In [ ]:
!pip install pandas-datareader


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_datareader as data 

In [ ]:
start = '2000-01-01'
end = '2020-12-31'

df=data.DataReader('AAPL', 'yahoo', start, end)
df.head()

In [ ]:
df.tail()

In [ ]:
df=df.reset_index()
df.head()

In [ ]:
df=df.drop(['Date','Adj Close' ] ,axis=1)
df.head()

In [ ]:
plt.plot(df.Close)

In [ ]:
df

In [ ]:
ma100=df.Close.rolling(100).mean()
ma100

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(df.Close)
plt.plot(ma100,'r')

In [ ]:
ma200=df.Close.rolling(200).mean()
ma200

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(df.Close)
plt.plot(ma100,'r')
plt.plot(ma200,'g')

In [ ]:
df.shape

In [ ]:
#slpiting training and testing
data_train=pd.DataFrame(df['Close'][0:int(len(df)*0.70)])
data_test=pd.DataFrame(df['Close'][int(len(df)*0.70):int(len(df))])

print(data_train.shape)
print(data_test.shape)

In [ ]:
data_train.head()

In [ ]:
data_test.head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))

In [ ]:
data_train_array=scaler.fit_transform(data_train)
data_train_array

In [ ]:
data_train_array.shape

In [ ]:
x_train=[]
y_train=[]

for i in range(100, data_train_array.shape[0]):
    x_train.append(data_train_array[i-100:i])
    y_train.append(data_train_array[i,0])
    
x_train

In [ ]:
y_train

In [ ]:
x_train=np.array(x_train)

In [ ]:
y_train=np.array(y_train)

In [ ]:
x_train

In [ ]:
x_train.shape

In [ ]:
x_train.size

In [ ]:
#ml model
      
!pip install keras

In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install tensorflow

In [ ]:
from keras.layers import Dense, Dropout, LSTM

In [ ]:
from keras.models import Sequential

In [ ]:
model=Sequential()
model.add(LSTM(units=50, activation='relu',return_sequences= True, input_shape=(x_train.shape[1],1)))
model.add(Dropout(0.2))

model.add(LSTM(units=60, activation='relu',return_sequences= True ))
model.add(Dropout(0.3))

model.add(LSTM(units=80, activation='relu',return_sequences= True))
model.add(Dropout(0.4))

model.add(LSTM(units=120, activation='relu'))
model.add(Dropout(0.5))
          
model.add(Dense(units=1))
          

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train,epochs=50)

In [ ]:
model.save('keras_model.h5')

In [ ]:
data_test.head()

In [ ]:
data_test.tail()

In [ ]:
past_100_days=data_train.tail(100)

In [ ]:
final_df=past_100_days.append(data_test, ignore_index=True)

In [ ]:
final_df.head()

In [ ]:
input_data=scaler.fit_transform(final_df)
input_data

In [ ]:
input_data.shape

In [ ]:
x_test=[]
y_test=[]

for i in range(100, input_data.shape[0]):
    x_test.append(input_data[i-100:i])
    y_test.append(input_data[i,0])

In [ ]:
x_test,y_test= np.array(x_test),np.array(y_test)
print(x_test.shape)
print(y_test.shape)

In [ ]:
y_predicted=model.predict(x_test)

In [ ]:
y_predicted.shape

In [ ]:
y_test

In [ ]:
y_predicted

In [ ]:
scaler.scale_

In [ ]:
scaler_factor=1/0.00859014
y_predicted=y_predicted*scaler_factor
y_test=y_test*scaler_factor

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(y_test,'b', label='Original Price')
plt.plot(y_predicted,'r',label='Predicted Price')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
plt.show()